In [21]:
import PySimpleGUI as sg

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from StyleFrame import StyleFrame
import smtplib,ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders
import re
from socket import gethostbyname,gaierror



layout = [[sg.Text('Выбрать файлы для рассылки')],
          [sg.Input("C:/документооборот/данные/Запрос в ТК.xlsx"), sg.FileBrowse("Запрос в ТК")],
          [sg.Input("C:/документооборот/данные/Контакты подрядчиков.xlsx"), sg.FileBrowse("Контакты")],
          [sg.Input("C:/документооборот/данные/КСЭ забор от водителей.xlsx"), sg.FileBrowse("КСЭ")],
          [sg.Input("C:/документооборот/файлы для контрагентов"), sg.FolderBrowse("Папка для выгрузки")],
          [sg.OK('начать рассылку'), sg.Cancel('отмена')]] 

window = sg.Window('e-mail delivery', layout)



def main():
    df = pd.read_excel(values[0]) 
    df_kse = pd.read_excel(values[2]) 
    
    #формируем таблицу с данными по КСЭ
    df_kse = df_kse[['Груз описание','Номер накладной КСЭ']]
    df_kse.dropna(inplace=True)
    df_kse = df_kse.loc[df_kse['Номер накладной КСЭ'].str.contains('496',flags=re.I,regex=True)]
    df_kse.columns= ['номер поездки','Номер накладной КСЭ']

    # фильтруем нужные значения, устанавливаем столбцы 
    df = df[(df['doc status']!="5")]
    df = df[['shp#','Subcontractor','zoek', 'delivery trip','load date','load address',
             'unload date','unload address','Driver','status comment']]
    df.columns = ['номер заявки','контрагент','клиент','номер поездки',
                  'дата загрузки','адрес загрузки','дата выгрузки','адрес выгрузки',
                 'водитель','комментарий']

    df.dropna(subset=['контрагент'],inplace=True) #дропаем те строки, где Subcontractor/контрагент не указан

    # собираем список уникальных контрагентов 
    partners = df['контрагент'].drop_duplicates()
    partners = partners.dropna()
    #убираем поездки, к которым уже высланы документы через КСЭ
    df= df.merge(df_kse,on='номер поездки', how='outer')
    df = df[pd.isnull(df['Номер накладной КСЭ'])==True]
    df.drop('Номер накладной КСЭ',axis=1,inplace=True)

    # выгружаем и редактируем 
    for i in range(len(partners)):
        write = df[df['контрагент']==partners.iloc[i]]
        excel_writer = StyleFrame.ExcelWriter(values[3]+'/'+str(partners.iloc[i])+'.xlsx')
        sf = StyleFrame(write)
        sf.set_column_width(columns=['дата загрузки', 'дата выгрузки'],
                            width=15.1)
        sf.set_column_width(columns=['адрес загрузки', 'адрес выгрузки','комментарий','водитель'],
                            width=40.1)
        sf.set_column_width(columns=['контрагент','клиент','номер заявки','номер поездки',],
                            width=15.1)
        sf.to_excel(excel_writer=excel_writer)
        excel_writer.save()

    emails = pd.read_excel(values[1])
    #формируем заголовок
    emails.columns = emails.iloc[5]
    emails = emails.iloc[6:]
    #оставляем нужные столбцы
    emails = emails[['Столбец1','Электронная почта','Электронная почта13']]
    emails.dropna(subset=['Столбец1'],inplace=True)
    emails.fillna('',inplace=True)
    emails['Столбец1'] = emails['Столбец1'].str.strip()
    emails['Электронная почта'] = emails['Электронная почта'].str.strip()
    emails['Электронная почта13'] = emails['Электронная почта13'].str.strip()
    emails['test_email'] ='empty'

    #ищем почтовые адресса
    for i in range(len(emails)):
        if '@' in emails['Электронная почта13'].iloc[i]:
            emails['test_email'].iloc[i] = emails['Электронная почта13'].iloc[i]
        else: 
            emails['test_email'].iloc[i] = emails['Электронная почта'].iloc[i]
    #удаляем дубликаты, сводим в одну строку
    emails['test_email'] = emails.groupby('Столбец1').transform(lambda x: ','.join(x))
    emails.drop_duplicates(subset=['Столбец1'],inplace=True)

    #удаляем лишние символы, которые попали в множество адресов
    for i in range(len(emails)):
        if '' in set(emails['test_email'].iloc[i].split(',')):
            emails['test_email'].iloc[i] = set(emails['test_email'].iloc[i].split(','))
            emails['test_email'].iloc[i].remove('')
        elif ' ' in set(emails['test_email'].iloc[i].split(',')):
            emails['test_email'].iloc[i] = set(emails['test_email'].iloc[i].split(','))
            emails['test_email'].iloc[i].remove(' ')
        else: 
            emails['test_email'].iloc[i] = set(emails['test_email'].iloc[i].split(','))

    #формируем таблицу с серчнеймами контрагентов
    partners = pd.DataFrame(partners)
    partners.rename(columns={'контрагент':'Столбец1'},inplace=True)
    partners['test']=1


    emails = emails.merge(partners,on='Столбец1', how='outer') 
    emails = emails[emails['test']==1.0] #пересечение нужных нам контрагентов в общей книге
    emails.fillna(0,inplace=True)

    without = emails[emails['Электронная почта']==0.0]['Столбец1'] #список контрагентов к которым нет контактов
    if len(without)>0:
        without.to_excel('нет контактов.xlsx',index=False) # выгружаем в файл
        sg.Popup('Сформирован файл с контрагентами, не занесенными в базу')
        
        
    emails = emails[emails['Электронная почта']!=0] # список контрагентов к которым есть почта      

    # функция отправляет письмо с одним файлом
    def send_one(send_from,send_to,subject,text,direction,username='',password='',isTls=True):
        msg = MIMEMultipart()
        msg['From'] = send_from
        msg['To'] =   send_to
        msg['Date'] = formatdate(localtime = True)
        msg['Subject'] = subject
        msg.attach(MIMEText(text))

        part = MIMEBase('application', "octet-stream")
        part.set_payload(open(direction, "rb").read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="report.xlsx"')
        msg.attach(part)

        smtp = smtplib.SMTP('smtp.gmail.com', 587)
        if isTls:
            smtp.starttls()
        smtp.login(username,password)
        smtp.sendmail(send_from, send_to, msg.as_string())
        smtp.quit()
    
    send_fail = pd.DataFrame(columns=('Столбец1','test_email')) #таблица, куда будут помещены адреса, по которым письма не ушли
   
    #Рассылка
    i=0
    while i < len(emails):
        j=0
        while j < len(emails['test_email'].iloc[i]):
            try:
                send_one('fm.mail.delivery@gmail.com',list(emails['test_email'].iloc[i])[j],\
                         subject='Нет документов('+emails['Столбец1'].iloc[i]+')',\
                         text=' Добрый день, коллеги.\n Во вложении список проблемных перевозок, просьба добавить свои комментарии в отдельном столбце.\n',\
                         username='fm.mail.delivery@gmail.com',password='*******',\
                         direction=(values[3]+'/'+emails['Столбец1'].iloc[i]+'.xlsx'))
            except (gaierror,ConnectionAbortedError,TimeoutError,smtplib.SMTPServerDisconnected) as err:
                e= sg.PopupYesNo('возникла ошибка соединения, повторить попытку?')
                if e =='Yes': continue
                elif e=='No' or e==None : #выгрузка контрагентов, на которых рассылка была остановлена
                    send_fail = send_fail.append(emails[['Столбец1','test_email']].iloc[i:], ignore_index=True)
                    break
            except:
                send_fail = send_fail.append({'Столбец1':emails['Столбец1'].iloc[i], 'test_email':list(emails['test_email'].iloc[i])[j]}, ignore_index=True)
            j+=1
        if e=='No' or e==None: break
        else: i+=1


    if len(send_fail) >0:
        send_fail.to_excel('недоставленные письма.xlsx',index=False) # файл с адресами по которым письма не ушли
        sg.Popup('Рассылка завершена, сформирован файл с недоставленными письмами')
    else:
        sg.Popup('Рассылка завершена')
    

while True:     
    try:
        event, values = window.Read()
        if event in (None,'отмена'):
            window.Close()
            break
        else:
            main()
            window.Close()
            break
    except:
        sg.Popup('Возникла ошибка')
        window.Close()
        break

